# core

> This is the core module : mostly data classes to manage Friends.

In [ ]:
#| default_exp core

In [ ]:
#| export

from datetime import datetime,date
from dateutil import parser as dtparse     
from dataclasses import dataclass
from uuid import uuid4

We write source code first, and then tests come after. The tests serve as both a means to confirm that the code works and also serves as working examples. The first exported function, parsed_date, is an example of this pattern.

In [ ]:
#| export
def parsed_date(s:str):
    "Convert `s` to a datetime"
    return dtparse.parse(s)

In [ ]:
parsed_date('2pm')

In [ ]:
isinstance(date.fromtimestamp(0), date)

The Friend dataclass : this is our main business object.

Note that I use [Python dataclass](https://docs.python.org/3/library/dataclasses.html) to define this class.

In [ ]:
#| export

@dataclass
class Friend:
    """Class for keeping track of a friend item."""
    id: str
    family_name: str
    first_name: str
    email: str
    phone: str
    linkedin: str
    twitter: str

In [ ]:
f1 = Friend(3, 'doo', 'john', 'john@doo.com', '0000000000', 'linkedin', 'twitter')
f1

In [ ]:
#| export

from fastsql import *
import sqlalchemy as sa

In [ ]:
db = Database("sqlite:///:memory:")

In [ ]:
friends1 = db.create(Friend, pk='id')

In [ ]:
print(db.schema())

In [ ]:
friends1.exists()

In [ ]:
friends1.insert(f1)

In [ ]:
friends1()

In [ ]:
friends1(where="family_name = :my", my="doo") 

After these tests, let's build our Repository service class.

In [ ]:
#| export

class Repository:
    """Repository Service for the Friend dataclass. 
    The repository implementation is based on SQLLite with a file storage"""
    
    def __init__(self, _db_file_path:str):
        """
        Constructor

        Parameters:
          _db_file_path: relative path of the file (to the script) 
        """
        self.db_file_path = _db_file_path
        self.create()

    def create(self):
        """
        Creates the database
        """
        uri = 'sqlite:///'+self.db_file_path
        self.db = Database(uri)
        self.store = self.db.create(Friend, pk='id')

    def add_obj(self, f:Friend):
        random_uuid = uuid4()
        """
        Adds a Friend object to the repository
        """
        self.store.insert(f)

    def add(self, family_name: str, 
            first_name: str,
            email: str,
            phone: str,
            linkedin: str,
            twitter: str):
        """
        Creates and Adds a Friend to the repository
        """
        random_uuid = uuid4() # str(random_uuid)
        friend = Friend(str(random_uuid), family_name, first_name, email, phone, linkedin, twitter)
        self.add_obj(friend)

    def friends(self):
        return self.store() 
    
    def __repr__(self):
        return "Repository SQLLite at " + self.db_file_path

Let's have an example here.

First, we create the repository.

In [ ]:
! rm friends_v1.db

In [ ]:
repo = Repository('friends_v1.db')
repo

You need to call the create method.

In [ ]:
repo.add('doo', 'john', 'john@doo.com', '0000000000', 'linkedin', 'twitter')

Now, let's retrieve my friends:

In [ ]:
repo.friends()

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()